In [1]:
import numpy as np
import pandas as pd

In [2]:
eventData = pd.read_csv('event.csv')
clickTrainData = pd.read_csv('click_train.csv')

dropEvent = ['timestamp', 'OS', 'browser', 'device', 'widgetId']
cleanEventData = eventData.drop(columns = dropEvent)

eventData = cleanEventData.sort_values(by = ['displayId']).to_numpy()
clickTrainData = clickTrainData.sort_values(by = ['displayId']).to_numpy()

(evn, evm) = eventData.shape
(trainn, trainm) = clickTrainData.shape

# merging displayId
columnsList = ['displayId', 'adId', 'clicked', 'pageId', 'userId']
(resn, resm) = (trainn, evm + trainm - 1)

result = np.zeros((resn, resm), dtype = np.int64)

curr = 0
for i in range(trainn):
    while curr < evn and eventData[curr][0] < clickTrainData[i][0]:
        curr += 1
        
    if curr < evn and clickTrainData[i][0] == eventData[curr][0]:
        cnt = 0
        for j in range(0, trainm):
            result[i][cnt] = clickTrainData[i][j]
            cnt += 1
        for j in range(1, evm):
            result[i][cnt] = eventData[curr][j]
            cnt += 1
    else:
        print('One missed row')

trainDisplayId = pd.DataFrame(data = result, columns = columnsList)
trainDisplayId.head()

,displayId,adId,clicked,pageId,userId
0,1210227,7182,0,420857,782749
1,1210227,7125,0,420857,782749
2,1210227,7181,0,420857,782749
3,1210227,535,0,420857,782749
4,1210227,7174,1,420857,782749


In [3]:
adTitle = pd.read_csv('ad_title.csv')

adTitle = adTitle.sort_values(by = ['adId']).to_numpy()
trainForAdId = trainDisplayId.sort_values(by = ['adId']).to_numpy()

numberOfWords = 5
(trainn, trainm) = trainForAdId.shape
(adn, adm) = adTitle.shape

# merging adId
columnsList = ['displayId', 'adId', 'clicked', 'pageId', 'userId']
for i in range(numberOfWords):
    columnsList.append('wordId' + str(i))
(resn, resm) = (trainn, trainm + numberOfWords)

result = np.zeros((resn, resm), dtype = np.int64)

curr = 0
for i in range(trainn):
    while curr < adn and adTitle[curr][0] < trainForAdId[i][1]:
        curr += 1
        
    if curr < adn and trainForAdId[i][1] == adTitle[curr][0]:
        cnt = 0
        for j in range(0, trainm):
            result[i][cnt] = trainForAdId[i][j]
            cnt += 1
        for j in range(numberOfWords):
            if adTitle[curr + j][0] == adTitle[curr][0]:
                result[i][cnt] = adTitle[curr + j][1]
            else:
                result[i][cnt] = 0 #Salam bar tof!
            cnt += 1
    else:
        cnt = 0
        for j in range(0, trainm):
            result[i][cnt] = trainForAdId[i][j]
            cnt += 1
        for j in range(numberOfWords):
            result[i][cnt] = 0 #Salam bar tof!
            cnt += 1

trainMerged = pd.DataFrame(data = result, columns = columnsList)
trainMerged = trainMerged.sort_values(by = ['displayId'])
trainMerged.head()

,displayId,adId,clicked,pageId,userId,wordId0,wordId1,wordId2,wordId3,wordId4
3642243,1210227,7182,0,420857,782749,108,675,734,1006,2313
3639160,1210227,7181,0,420857,782749,853,1006,734,675,108
745066,1210227,535,0,420857,782749,13,16,135,859,0
3625504,1210227,7125,0,420857,782749,2544,1221,912,910,333
3636967,1210227,7174,1,420857,782749,29,109,203,2571,2572


In [4]:
pageTopicData = pd.read_csv('page_topic.csv')

trainTopic = trainMerged.sort_values(by = ['pageId']).to_numpy()
pageTopicData = pageTopicData.sort_values(by = ['pageId']).to_numpy()

(ton, tom) = pageTopicData.shape
(trainn, trainm) = trainTopic.shape

# only add topicId
columnsList = ['displayId', 'adId', 'clicked', 'pageId', 'userId']
for i in range(numberOfWords):
    columnsList.append('wordId' + str(i))
columnsList.append('topicId')
(resn, resm) = (trainn, trainm + 1)

result = np.zeros((resn, resm), dtype = np.int64)

curr = 0
for i in range(trainn):
    while curr < ton and pageTopicData[curr][0] < trainTopic[i][3]:
        curr += 1
        
    if curr < ton and trainTopic[i][3] == pageTopicData[curr][0]:
        maxId = curr
        k = curr + 1
        while k < ton and pageTopicData[k][0] == pageTopicData[curr][0]:
            if pageTopicData[k][2] > pageTopicData[maxId][2]:
                maxId = k
            k += 1
            
        cnt = 0
        for j in range(0, trainm):
            result[i][cnt] = trainTopic[i][j]
            cnt += 1
        result[i][cnt] = pageTopicData[maxId][1]
    else:
        cnt = 0
        for j in range(0, trainm):
            result[i][cnt] = trainTopic[i][j]
            cnt += 1

trainMergedAll = pd.DataFrame(data = result, columns = columnsList)
trainMergedAll = trainMergedAll.sort_values(by = ['displayId'])
trainMergedAll.head()

,displayId,adId,clicked,pageId,userId,wordId0,wordId1,wordId2,wordId3,wordId4,topicId
880284,1210227,535,0,420857,782749,13,16,135,859,0,20
880281,1210227,7182,0,420857,782749,108,675,734,1006,2313,20
880282,1210227,7174,1,420857,782749,29,109,203,2571,2572,20
880283,1210227,7125,0,420857,782749,2544,1221,912,910,333,20
880285,1210227,7181,0,420857,782749,853,1006,734,675,108,20


In [5]:
train = trainMergedAll.copy(deep = True)
train = train.to_numpy()
y_train = []
X_train = []
for i in range(train.shape[0]):
    y_train.append(train[i][2])
    X_train.append([*train[i][:2], *train[i][3:]])
    
columnsList = ['displayId', 'adId', 'pageId', 'userId']
for i in range(numberOfWords):
    columnsList.append('wordId' + str(i))
columnsList.append('topicId')
    
X_train = pd.DataFrame(data = X_train, columns = columnsList)
y_train = pd.DataFrame(data = y_train, columns = ['clicked'])

In [6]:
X_trainWithoutIds = X_train.drop(columns = ['displayId', 'adId'])

In [16]:
X_trainWithoutIds.head()

,pageId,userId,wordId0,wordId1,wordId2,wordId3,wordId4,topicId
0,420857,782749,13,16,135,859,0,20
1,420857,782749,108,675,734,1006,2313,20
2,420857,782749,29,109,203,2571,2572,20
3,420857,782749,2544,1221,912,910,333,20
4,420857,782749,853,1006,734,675,108,20


In [7]:
from sklearn.tree import DecisionTreeClassifier
treeModel = DecisionTreeClassifier(random_state=1, max_depth=15)
treeModel.fit(X_trainWithoutIds, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=15, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=1, splitter='best')

In [8]:
eventData = pd.read_csv('event.csv')
clickTrainData = pd.read_csv('click_test.csv')

dropEvent = ['timestamp', 'OS', 'browser', 'device', 'widgetId']
cleanEventData = eventData.drop(columns = dropEvent)

eventData = cleanEventData.sort_values(by = ['displayId']).to_numpy()
clickTrainData = clickTrainData.sort_values(by = ['displayId']).to_numpy()

(evn, evm) = eventData.shape
(trainn, trainm) = clickTrainData.shape

# merging displayId
columnsList = ['displayId', 'adId', 'pageId', 'userId']
(resn, resm) = (trainn, evm + trainm - 1)

result = np.zeros((resn, resm), dtype = np.int64)

curr = 0
for i in range(trainn):
    while curr < evn and eventData[curr][0] < clickTrainData[i][0]:
        curr += 1
        
    if curr < evn and clickTrainData[i][0] == eventData[curr][0]:
        cnt = 0
        for j in range(0, trainm):
            result[i][cnt] = clickTrainData[i][j]
            cnt += 1
        for j in range(1, evm):
            result[i][cnt] = eventData[curr][j]
            cnt += 1
    else:
        print('One missed row')

trainDisplayId = pd.DataFrame(data = result, columns = columnsList)
trainDisplayId.head()

,displayId,adId,pageId,userId
0,151650,7585,4095035,10279
1,151650,6257,4095035,10279
2,151650,6690,4095035,10279
3,151938,7715,4273336,17723
4,151938,6690,4273336,17723


In [9]:
adTitle = pd.read_csv('ad_title.csv')

adTitle = adTitle.sort_values(by = ['adId']).to_numpy()
trainForAdId = trainDisplayId.sort_values(by = ['adId']).to_numpy()

numberOfWords = 5
(trainn, trainm) = trainForAdId.shape
(adn, adm) = adTitle.shape

# merging adId
columnsList = ['displayId', 'adId', 'pageId', 'userId']
for i in range(numberOfWords):
    columnsList.append('wordId' + str(i))
(resn, resm) = (trainn, trainm + numberOfWords)

result = np.zeros((resn, resm), dtype = np.int64)

curr = 0
for i in range(trainn):
    while curr < adn and adTitle[curr][0] < trainForAdId[i][1]:
        curr += 1
        
    if curr < adn and trainForAdId[i][1] == adTitle[curr][0]:
        cnt = 0
        for j in range(0, trainm):
            result[i][cnt] = trainForAdId[i][j]
            cnt += 1
        for j in range(numberOfWords):
            if adTitle[curr + j][0] == adTitle[curr][0]:
                result[i][cnt] = adTitle[curr + j][1]
            else:
                result[i][cnt] = 0 #Salam bar tof!
            cnt += 1
    else:
        cnt = 0
        for j in range(0, trainm):
            result[i][cnt] = trainForAdId[i][j]
            cnt += 1
        for j in range(numberOfWords):
            result[i][cnt] = 0 #Salam bar tof!
            cnt += 1

trainMerged = pd.DataFrame(data = result, columns = columnsList)
trainMerged = trainMerged.sort_values(by = ['displayId'])
trainMerged.head()

,displayId,adId,pageId,userId,wordId0,wordId1,wordId2,wordId3,wordId4
468220,151650,6690,4095035,10279,2487,2488,574,148,364
1117826,151650,7585,4095035,10279,57,151,202,605,0
355417,151650,6257,4095035,10279,602,76,2364,0,0
35592,151938,123,4273336,17723,21,224,340,341,0
1017470,151938,7379,4273336,17723,1849,108,1848,0,0


In [11]:
pageTopicData = pd.read_csv('page_topic.csv')

trainTopic = trainMerged.sort_values(by = ['pageId']).to_numpy()
pageTopicData = pageTopicData.sort_values(by = ['pageId']).to_numpy()

(ton, tom) = pageTopicData.shape
(trainn, trainm) = trainTopic.shape

# only add topicId
columnsList = ['displayId', 'adId', 'pageId', 'userId']
for i in range(numberOfWords):
    columnsList.append('wordId' + str(i))
columnsList.append('topicId')
(resn, resm) = (trainn, trainm + 1)

result = np.zeros((resn, resm), dtype = np.int64)

curr = 0
for i in range(trainn):
    while curr < ton and pageTopicData[curr][0] < trainTopic[i][2]:
        curr += 1
        
    if curr < ton and trainTopic[i][2] == pageTopicData[curr][0]:
        maxId = curr
        k = curr + 1
        while k < ton and pageTopicData[k][0] == pageTopicData[curr][0]:
            if pageTopicData[k][2] > pageTopicData[maxId][2]:
                maxId = k
            k += 1
            
        cnt = 0
        for j in range(0, trainm):
            result[i][cnt] = trainTopic[i][j]
            cnt += 1
        result[i][cnt] = pageTopicData[maxId][1]
    else:
        cnt = 0
        for j in range(0, trainm):
            result[i][cnt] = trainTopic[i][j]
            cnt += 1

trainMergedAll = pd.DataFrame(data = result, columns = columnsList)
trainMergedAll = trainMergedAll.sort_values(by = ['displayId'])
trainMergedAll.head()

,displayId,adId,pageId,userId,wordId0,wordId1,wordId2,wordId3,wordId4,topicId
420051,151650,7585,4095035,10279,57,151,202,605,0,28
420050,151650,6690,4095035,10279,2487,2488,574,148,364,28
420052,151650,6257,4095035,10279,602,76,2364,0,0,28
438263,151938,6690,4273336,17723,2487,2488,574,148,364,8
438268,151938,7379,4273336,17723,1849,108,1848,0,0,8


In [17]:
X_test = trainMergedAll[['pageId', 'userId', 'wordId0', 'wordId1', 'wordId2', 'wordId3', 'wordId4', 'topicId']]
X_test.head()

,pageId,userId,wordId0,wordId1,wordId2,wordId3,wordId4,topicId
420051,4095035,10279,57,151,202,605,0,28
420050,4095035,10279,2487,2488,574,148,364,28
420052,4095035,10279,602,76,2364,0,0,28
438263,4273336,17723,2487,2488,574,148,364,8
438268,4273336,17723,1849,108,1848,0,0,8


In [18]:
predictions = treeModel.predict_proba(X_test)

In [21]:
def rankList(probabilities):
    ans = []
    n = len(probabilities)
    for i in range(n):
        cnt = 1
        for j in range(n):
            if i == j:
                continue
            if probabilities[i] < probabilities[j] or (probabilities[i] == probabilities[j] and j < i):
                cnt += 1
        ans.append(cnt)
    return ans

In [22]:
testData = pd.read_csv('click_test.csv').to_numpy()
testPre = trainMergedAll.to_numpy()

(n, m) = testData.shape
(tn, tm) = testPre.shape

rank = []

i = 0

l = 0
while l < tn:
    r = l + 1
    while r < tn and testPre[r][0] == testPre[l][0]:
        r += 1
    List = []
    lki = i
    while i < n and testData[i][0] == testPre[l][0]:
        for j in range(l, r):
            if testPre[j][1] == testData[i][1]:
                List.append(predictions[j][1])
                break
        i += 1
    rankListed = rankList(List)
    for j in range(lki, i):
        rank.append(rankListed[j - lki])
    l = r
Answer = pd.DataFrame(data = testData, columns = ['displayId', 'adId'])
Answer['clicked'] = rank
Answer.head()

,displayId,adId,clicked
0,151650,7585,2
1,151650,6257,1
2,151650,6690,3
3,151938,7454,1
4,151938,7370,6


In [23]:
Answer.to_csv('answerWith5WordsTopic.csv',index=False)